In [107]:
# gpu number setting
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1' ## gpu 번호 셋팅 윤건 :0, 기준 : 1, 준형 :2,

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.3.1
keras version :  2.4.0
GPU available ? :  True


In [108]:
import pandas as pd
import tqdm
import numpy as np

In [109]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [141]:
raw = pd.read_csv('../../Data/movie_201118.csv')

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,26,27,28,29,37,39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


* null 있지만 채울 필요 없는 것들,

e_time

watch_duration

vod_s_point, vod_e_point

agree_yn

category_id (too many)

### buy_history_price 조정 필요

* need to fill

release_date (median)

In [142]:
raw['release_date'].min()

19350101.0

In [143]:
raw['release_date'].max()

20201113.0

In [144]:
raw['release_date'].median()

20170428.0

In [145]:
raw['release_date'].fillna(raw['release_date'].median(), inplace=True)

meta_genre, genre_large, genre_mid, genre_small ('N')

In [146]:
genre_col = ['meta_genre', 'genre_large', 'genre_mid', 'genre_small']

In [147]:
for col in genre_col:
    print(sum(raw[col].isnull()))

8347
5275
5275
7229081


In [148]:
for col in genre_col:
    raw[col].fillna('N', inplace=True)

In [149]:
for col in genre_col:
    print(sum(raw[col].isnull()))

0
0
0
0


buy_seg

buy_seg = '\\N' 인 것들 중에,

if__  buy_history_price 까지 '\\N' or 0원 이면 : fod

else__ price > 0 이면 : 'BUY_1_2'

## 로 하려고하는데, buy_seg = '\\N' 이면 amt_1_4 - amt_4_4 모두 '\\N' 이라 일단 뺌.

In [153]:
raw.drop(raw[raw['buy_seg'] == '\\N'].index, inplace=True)

In [157]:
print(sum(raw['buy_seg'] == '\\N'))
print(sum(raw['amt_1_4'] == '\\N'))
print(sum(raw['amt_2_4'] == '\\N'))
print(sum(raw['amt_3_4'] == '\\N'))
print(sum(raw['amt_4_4'] == '\\N'))

0
0
0
0
0


In [130]:
# raw.loc[(raw['buy_seg'] == '\\N') & (raw['buy_history_price'].apply(lambda x: x in ['\\N', 0])), 'buy_seg'] = 'FOD'

In [131]:
# raw.loc[raw['buy_seg'] == '\\N', 'buy_seg'] = 'BUY_1_2'

In [158]:
set(raw['buy_seg'])

{'BUY_1_2', 'BUY_3', 'FOD', 'VIEW_NO'}

In [159]:
raw[raw['amt_1_4'] == '\\N']

,sa_id,s_time,e_time,watch_duration,album_id,album_name,gubun,vod_s_point,vod_e_point,agree_yn,category_id,release_date,run_time,meta_genre,genre_large,genre_mid,genre_small,movie_meta_price,buy_history_price,i30_meta_price,buy_tot,view_no,fod,buy_1_2,buy_3,buy_seg,amt_1_4,amt_2_4,amt_3_4,amt_4_4,amt_r_gabun,weekdays,weekends,dawn,morning,afternoon,evening,watch_ratio,current_rate,ncf_rating


In [161]:
raw[raw['amt_2_4'] == '\\N']

,sa_id,s_time,e_time,watch_duration,album_id,album_name,gubun,vod_s_point,vod_e_point,agree_yn,category_id,release_date,run_time,meta_genre,genre_large,genre_mid,genre_small,movie_meta_price,buy_history_price,i30_meta_price,buy_tot,view_no,fod,buy_1_2,buy_3,buy_seg,amt_1_4,amt_2_4,amt_3_4,amt_4_4,amt_r_gabun,weekdays,weekends,dawn,morning,afternoon,evening,watch_ratio,current_rate,ncf_rating


In [162]:
raw[raw['amt_3_4'] == '\\N']

,sa_id,s_time,e_time,watch_duration,album_id,album_name,gubun,vod_s_point,vod_e_point,agree_yn,category_id,release_date,run_time,meta_genre,genre_large,genre_mid,genre_small,movie_meta_price,buy_history_price,i30_meta_price,buy_tot,view_no,fod,buy_1_2,buy_3,buy_seg,amt_1_4,amt_2_4,amt_3_4,amt_4_4,amt_r_gabun,weekdays,weekends,dawn,morning,afternoon,evening,watch_ratio,current_rate,ncf_rating


In [163]:
raw[raw['amt_4_4'] == '\\N']

,sa_id,s_time,e_time,watch_duration,album_id,album_name,gubun,vod_s_point,vod_e_point,agree_yn,category_id,release_date,run_time,meta_genre,genre_large,genre_mid,genre_small,movie_meta_price,buy_history_price,i30_meta_price,buy_tot,view_no,fod,buy_1_2,buy_3,buy_seg,amt_1_4,amt_2_4,amt_3_4,amt_4_4,amt_r_gabun,weekdays,weekends,dawn,morning,afternoon,evening,watch_ratio,current_rate,ncf_rating


In [177]:
raw.reset_index(inplace=True)

In [178]:
raw

,index,sa_id,s_time,e_time,watch_duration,album_id,album_name,gubun,vod_s_point,vod_e_point,agree_yn,category_id,release_date,run_time,meta_genre,genre_large,genre_mid,genre_small,movie_meta_price,buy_history_price,i30_meta_price,buy_tot,view_no,fod,buy_1_2,buy_3,buy_seg,amt_1_4,amt_2_4,amt_3_4,amt_4_4,amt_r_gabun,weekdays,weekends,dawn,morning,afternoon,evening,watch_ratio,current_rate,ncf_rating
0,0,205101491713,20200826090744,2.020083e+13,6676.0,M01208D595PPV00,반도(소장),WT,40.0,6716.0,Y,NaN,20200715.0,115,액션,영화,액션,N,9900.0,\N,9900,0,1,0,0,0,VIEW_NO,1,0,0,0,AMT_0_R,1,0,0,1,0,0,0.967536231884058,0.461111,0.7532125603864734
1,1,205101521683,20200827211322,NaN,\N,M011526134PPV00,[우리말]마다가스카,W,NaN,NaN,Y,NaN,20050714.0,85,극장판 애니,영화,극장판 애니,어린이/가족,1400.0,\N,1400,0,0,1,0,0,FOD,1,0,0,0,AMT_0_R,1,0,0,0,0,1,0.8,0.455556,0.6722222222222223
2,2,205101521683,20201001155631,2.020100e+13,5193.0,M01208A459PPV00,[우리말]메가마인드,WT,25.0,5218.0,Y,NaN,20110113.0,95,극장판 애니,영화,극장판 애니,어린이/가족,1400.0,\N,1400,0,0,1,0,0,FOD,1,0,0,0,AMT_0_R,1,0,0,0,1,0,0.9110526315789473,0.261111,0.8249707602339181
3,3,205101521683,20201103103535,2.020110e+13,6293.0,M01188R075PPV00,기문둔갑,WT,33.0,6326.0,Y,NaN,20171215.0,112,액션,영화,액션,N,1400.0,\N,1400,0,0,1,0,0,FOD,1,0,0,0,AMT_0_R,1,0,0,1,0,0,0.9364583333333333,0.077778,0.9293402777777777
4,4,205101521683,20200925212911,2.020093e+13,110.0,M01133TA30PPV00,가디언즈,WT,140.0,250.0,Y,NaN,20121129.0,97,극장판 애니,영화,극장판 애니,어린이/가족,1400.0,\N,1400,0,0,1,0,0,FOD,1,0,0,0,AMT_0_R,1,0,0,0,0,1,0.018900343642611683,0.294444,0.3622279495990836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8459259,8459359,M18061900001,20200909211937,NaN,\N,M012054529PPV00,[우리말]마다가스카 3: 이번엔 서커스다!(소장),W,NaN,NaN,Y,AL093,20120606.0,94,극장판 애니,영화,극장판 애니,어린이/가족,13900.0,\N,13900,8,0,0,0,1,BUY_3,0.0,0.375,0.0,0.625,AMT_5000_OVER_R,1,0,0,0,0,1,0.8,0.383333,0.708333
8459260,8459360,M18061900001,20200922130942,2.020092e+13,30.0,M01204M075PPV00,양자물리학,WT,338.0,368.0,Y,NaN,20190925.0,120,스릴러,영화,스릴러,N,3500.0,3500,3500,8,0,0,0,1,BUY_3,0.0,0.375,0.0,0.625,AMT_5000_OVER_R,1,0,0,0,1,0,0.00416667,0.311111,0.346528
8459261,8459361,M18061900001,20201001230620,2.020100e+13,6116.0,M01206U082PPV00,나의 첫 번째 슈퍼스타(소장),WT,677.0,6793.0,Y,NaN,20200610.0,113,코미디,영화,코미디,N,7700.0,9900,7700,8,0,0,0,1,BUY_3,0.0,0.375,0.0,0.625,AMT_5000_OVER_R,1,0,0,0,0,1,0.902065,0.261111,0.820477
8459262,8459362,M18061900001,20201001230620,2.020100e+13,6116.0,M01206U082PPV00,나의 첫 번째 슈퍼스타(소장),WT,-6116.0,0.0,Y,NaN,20200610.0,113,코미디,영화,코미디,N,7700.0,9900,7700,8,0,0,0,1,BUY_3,0.0,0.375,0.0,0.625,AMT_5000_OVER_R,1,0,0,0,0,1,0.902065,0.261111,0.820477


In [172]:
import pickle

In [179]:
with open('../../Data/movie_201118_table_2.pickle', 'wb') as f:
    pickle.dump(raw, f)